In [1]:
import time
import torch
from torch.utils.data import Dataset
from torch.utils.data import Subset
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.autograd import Variable
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import scipy.io
import numpy as np
import math
from sklearn import svm
from sklearn import metrics
import math
import time
import matplotlib.pyplot as plt


In [2]:
data = torchvision.datasets.ImageFolder("extracted_images")

In [3]:
def create_data(data):
    x = []
    y = []
    for tup in data.imgs:
        path = tup[0]
        label = tup[1]
        image = Image.open(path)
        converter = torchvision.transforms.ToTensor()
        x.append(converter.__call__(image).numpy())
        y.append(label)
    x = np.array(x)
    y = np.array(y)
    return x,y
        

In [4]:
x,y = create_data(data)

In [5]:
class HandWrittenDataset(Dataset):
    
    # TODO:
    # Define constructor for AnimalDataset class
    # HINT: You can pass processed data samples and their ground truth values as parameters 
    def __init__(self, x,y):
        super(HandWrittenDataset, self).__init__()
        self.data = x
        self.truth = y
    '''This function should return sample count in the dataset'''
    def __len__(self):
        return self.data.shape[0]

    '''This function should return a single sample and its ground truth value from the dataset corresponding to index parameter '''
    def __getitem__(self, index):
        _x = self.data[index]
        _y = self.truth[index]
        return _x, _y

In [6]:
all_data = HandWrittenDataset(x,y)
x.shape[0]
print(all_data.__len__)

<bound method HandWrittenDataset.__len__ of <__main__.HandWrittenDataset object at 0x108fb0b00>>


In [7]:
train_dataset,test_dataset = torch.utils.data.random_split(all_data , [300779, 75195])

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
class Softmax(nn.Module):
    '''Define your neural network'''
    def __init__(self, **kwargs): 
          super(Softmax, self).__init__()
          self.linear1 = nn.Linear(2025,82,bias=True).to(device)
    def forward(self, X): 
        X = self.linear1(X.view(X.shape[0],-1))
        return X

In [10]:
def correctPred(preds,truths):
    correct = 0
    for prediction,truth in zip(preds,truths):
        pred = np.argmax(prediction)
        if pred == truth:
            correct = correct + 1
    return correct

In [11]:
class MLP2(nn.Module):
    '''Define your neural network'''
    def __init__(self, **kwargs): 
          super(MLP2, self).__init__()
          self.linear1 = nn.Linear(2025,1000,bias=True).to(device)
          self.linear2 = nn.Linear(1000,250,bias=True).to(device)
          self.linear3 = nn.Linear(250,82,bias=True).to(device)
     
    def forward(self, X): 
        X = self.linear1(X.view(X.shape[0],-1))
        X = F.relu(X)
        X = self.linear2(X)
        X = F.relu(X)
        X = self.linear3(X)
        return X

In [ ]:
class ConvNet(nn.Module):
    '''Define your neural network'''
    def __init__(self, **kwargs): # you can add any additional parameters you want 
    # TODO:
    # You should create your neural network here
    # TODO:
      super(ConvNet, self).__init__()
      self.conv1 = nn.Conv2d(1, 8, kernel_size=3,stride = 1).to(device)
      self.conv2 = nn.Conv2d(8, 16, kernel_size=3,stride=1).to(device)
      self.conv3 = nn.Conv2d(16, 32, kernel_size=3,stride=1).to(device)
      self.mp = nn.MaxPool2d(2)
      self.fc = nn.Linear(2592, 82).to(device) 
     
    def forward(self, X): # you can add any additional parameters you want
    # TODO:
    # Forward propagation implementation should be here
      length = X.shape[0]
      X = self.conv1(X)
      X = F.relu(X)
      X = self.mp(self.conv2(X))
      X = F.relu(X)
      X = self.mp(self.conv3(X))
      X = F.relu(X)
      output = self.fc(X.view(X.size(0),-1))
      return output

In [18]:
#HINT: note that your training time should not take many days.

#TODO:
#Pick your hyper parameters
max_epoch = 250
train_batch = 50
test_batch = 100

learning_rate = 0.001

def main(train_dataset,test_dataset): # you are free to change parameters
    trainloader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=train_batch,
                                           shuffle=True)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=test_batch,
                                           shuffle=False)
    # initialize your GENet neural network
    model = MLP2()
    # define your loss function
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate,momentum=0.9,weight_decay=5e-04) # you can play with momentum and weight_decay parameters as well
    train_loss = []
    test_loss = []
    test_acc = []
    best_path = "MLP3.pth"
    best_acc = 0
    for epoch in range(max_epoch):
      train(epoch, model, criterion, optimizer,trainloader,train_loss)
      acc = test(model, criterion, test_loader,test_loss,test_acc)
      if acc > best_acc:
        best_acc = acc
        torch.save(model,best_path)
    return model,train_loss,test_loss,test_acc
''' Train your network for a one epoch '''
def train(epoch, model, criterion, optimizer,loader,train_loss): # you are free to change parameters
    model.train()
    loss_total = []
    for batch_idx, (data, labels) in enumerate(loader):
        # TODO:
        # Implement training code for a one iteration
        #data_time means data loading time per batch, and batch_time means forward/backward time during training.
        data = data.to(device)
        labels = labels.to(device).flatten()
        data, labels= Variable(data), Variable(labels)#might be unnecessary
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        loss_total.append(loss.data)
    
    print("Epoch: ",epoch, " loss: ",np.mean(np.array(loss_total)))
    train_loss.append(np.mean(np.array(loss_total)))

''' Test&Validate your network '''
def test(model, criterion, loader,test_loss,test_acc): # you are free to change parameters
    total = 0
    correct = 0
    count = 0
    total_loss = 0
    model.eval()
    with torch.no_grad():
        for batch_idx, (data, labels) in enumerate(loader):
            # TODO:
            # Implement test code
            count = count+1
            data = data.to(device)
            labels = labels.to(device).flatten()
            data, labels= Variable(data), Variable(labels)
            output = model(data)
            total = total + output.shape[0]
            correct = correct + correctPred(output,labels)
            loss = criterion(output,labels)
            total_loss = total_loss + loss.data
    test_loss.append(total_loss/count)
    test_acc.append(correct/total)
    return correct / total

In [ ]:
trained_model,train_loss,test_loss,test_acc = main(train_dataset,test_dataset)

Epoch:  0  loss:  2.7149465
Epoch:  1  loss:  1.5265262
Epoch:  2  loss:  1.1155239
Epoch:  3  loss:  0.90034807
Epoch:  4  loss:  0.7647346
Epoch:  5  loss:  0.67289037
Epoch:  6  loss:  0.6079046
Epoch:  7  loss:  0.55604464
Epoch:  8  loss:  0.5198705
Epoch:  9  loss:  0.48412582
Epoch:  10  loss:  0.45649776
Epoch:  11  loss:  0.42933556
Epoch:  12  loss:  0.4080041
Epoch:  13  loss:  0.38822722
Epoch:  14  loss:  0.37231737
Epoch:  15  loss:  0.35539085
Epoch:  16  loss:  0.341876
Epoch:  17  loss:  0.32706535
Epoch:  18  loss:  0.31730798
Epoch:  19  loss:  0.30743992
Epoch:  20  loss:  0.2948181
Epoch:  21  loss:  0.28677672
Epoch:  22  loss:  0.27742305
Epoch:  23  loss:  0.2709831
Epoch:  24  loss:  0.2613363
Epoch:  25  loss:  0.25529188
Epoch:  26  loss:  0.24956279
Epoch:  27  loss:  0.24360178
Epoch:  28  loss:  0.237392
Epoch:  29  loss:  0.23224242
Epoch:  30  loss:  0.22633481
Epoch:  31  loss:  0.22176613
Epoch:  32  loss:  0.21645635
Epoch:  33  loss:  0.21241693
Epoc

In [ ]:
plt.plot(range(len(train_loss)),train_loss , label='Training loss')
plt.plot(range(len(test_loss)),test_loss , label='Test loss')
plt.xlabel('Epoch ')
plt.ylabel('Loss')
plt.title('Loss vs Epoch MLP3')
plt.legend()
plt.show()

In [ ]:
plt.plot(range(len(test_acc)),test_acc,label='Test accuracy')
plt.xlabel('Epoch ')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Epoch MLP2')
plt.legend()
plt.show()

In [5]:
import scipy.io
import numpy as np
import math
from sklearn import svm
from sklearn import metrics
import math
import time
import matplotlib.pyplot as plt
import sklearn
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(np.reshape(x,(375974,-1)),y)
clf = svm.SVC(kernel='linear', C=0.1)


In [ ]:
clf.fit(X_train,y_train)
predict = clf.predict(X_test)
correct = correctPred(predict,y_test)
print("Accuracy: " , correct/len(y_test))

In [ ]:
max(test_acc)

In [11]:
x.shape

(375974, 1, 45, 45)